In [6]:
# importing dependencies
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from tkinter import *
from tkinter.ttk import Scale
from tkinter import colorchooser,filedialog,messagebox
import PIL.ImageGrab as ImageGrab
from PIL import Image
import cv2

In [7]:
# MNIST code
#device config
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'

#hyper parameters
input_size = 784
hidden_size_1 = 512
hidden_size_2 = 256
num_classes = 10
num_epochs = 3
batch_size = 200 
learning_rate = 0.001


#MNIST
train_dataset = torchvision.datasets.MNIST(root='./data', train = True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train = False, transform=transforms.ToTensor(), download=False)

train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = False)

# if __name__ == 'main':
examples = iter(train_loader)
samples, labels = next(examples)
# print(samples.shape, labels.shape)

#View dataset
# for i in range(6):
#     plt.subplot(2,3,i+1)
#     plt.imshow(samples[i][0], cmap='gray')
#     print(samples.shape)
# plt.show()

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size_1,hidden_size_2, num_classes) :
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size_1)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size_1, hidden_size_2)
        self.sigmoid = nn.Sigmoid()
        self.l3 = nn.Linear(hidden_size_2, num_classes)
        
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.sigmoid(out)
        out = self.l3(out)
        return out
    # We are not using softmax because we will use Cross Entropy Loss which automatically uses Softmax

model = NeuralNet(input_size, hidden_size_1,hidden_size_2, num_classes).to(device)

#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# if __name__ == 'main':
#training loop
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        #100, 1, 28, 28
        #100, 784
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)

        #forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        #backwards
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if(i+1)%100 == 0:
            print(f"Epoch {epoch +1}/{num_epochs}, step{i+1}/{n_total_steps}, loss = {loss.item():.4f}")

#test
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)

        # value, index
        _, predictions = torch.max(outputs, 1)
        n_samples +=labels.shape[0]
        n_correct += (predictions == labels).sum().item()

    acc = 100.0 * n_correct/ n_samples
    print(f'accuracy = {acc}')


Epoch 1/3, step100/300, loss = 0.2673
Epoch 1/3, step200/300, loss = 0.3537
Epoch 1/3, step300/300, loss = 0.1631
Epoch 2/3, step100/300, loss = 0.2382
Epoch 2/3, step200/300, loss = 0.0925
Epoch 2/3, step300/300, loss = 0.1500
Epoch 3/3, step100/300, loss = 0.1173
Epoch 3/3, step200/300, loss = 0.1147
Epoch 3/3, step300/300, loss = 0.0835
accuracy = 97.41


In [8]:
samples.shape

torch.Size([200, 1, 28, 28])

In [9]:
def softmax(x):
    return np.exp(x)/(np.sum(np.exp(x),axis = 1, keepdims = True))

def identify():
    image = Image.open('temp.jpg').convert('L')
    data = 255 - np.asarray(image)
#     plt.subplot(1,2,1)
    res = cv2.resize(data, dsize=(28, 28), interpolation=cv2.INTER_AREA )
#     plt.imshow(res)
    print(type(res))
    res = res.reshape(1,28*28)/255
#     torchvision.transforms.functional.to_tensor
    res = torch.from_numpy(res).float()
    ans = model(res).detach().numpy()
    ans = softmax(ans)
    ans = np.argmax(ans)
#     print(res.shape)
#     plt.subplot(1,2,2)
#     plt.imshow(data)
#     if messagebox.askquestion("Number guesser" , "Did you draw" + str(ans)):
#         if messagebox.askretrycancel():
#             identify()
#     else:
#         if messagebox.askretrycancel("Do you want to try again"):
#             identify()
    print(ans)
identify()

<class 'numpy.ndarray'>
6


In [16]:
# Digit drawing app    
class Draw():
    def __init__(self,root):
#Defining title and Size of the Tkinter Window GUI
        self.root =root
        self.root.title("Digit Drawing Tool")
        self.root.geometry("400x350")
        self.root.configure(background="white")
        # self.root.resizable(0,0)
 
#variables for pointer and Eraser   
        self.pointer= "black"
        self.erase="white"

#Widgets for Tkinter Window
    
# Configure the alignment , font size and color of the text
        text=Text(root)
        text.tag_configure("tag_name", justify='center', font=('arial',25),background='#292826',foreground='orange')

# Insert a Text
        text.insert("1.0", "Draw only one digit (0-9)")

# Add the tag for following given text
        text.tag_add("tag_name", "1.0", "end")
        text.pack()
        
 # Erase Button and its properties   
        self.eraser_btn= Button(self.root,text="Eraser",bd=4,bg='white',command=self.eraser,width=9,relief=RIDGE)
        self.eraser_btn.place(x=0,y=40)

# Reset Button to clear the entire screen 
        self.clear_screen= Button(self.root,text="Clear Screen",bd=4,bg='white',command= lambda : self.background.delete('all'),width=9,relief=RIDGE)
        self.clear_screen.place(x=0,y=70)

# Save Button for saving the image in local computer
        self.save_btn= Button(self.root,text="Identify",bd=4,bg='white',command=self.save_drawing,width=9,relief=RIDGE)
        self.save_btn.place(x=0,y=100)

# Background Button for choosing color of the Canvas
        # self.bg_btn= Button(self.root,text="Background",bd=4,bg='white',command=self.canvas_color,width=9,relief=RIDGE)
        # self.bg_btn.place(x=0,y=287)


#Creating a Scale for pointer and eraser size
        self.pointer_frame= LabelFrame(self.root,text='size',bd=5,bg='white',font=('arial',15,'bold'),relief=RIDGE)
        self.pointer_frame.place(x=0,y=130,height=200,width=70)

        self.pointer_size =Scale(self.pointer_frame,orient=VERTICAL,from_ =48 , to =0, length=168)
        self.pointer_size.set(25)
#         self.pointer_size.grid(row=0,column=1,padx=15)

#Defining a background color for the Canvas 
        self.background = Canvas(self.root,bg='white',bd=5,relief=GROOVE,height=470,width=680)
        self.background.place(x=80,y=40)

#Bind the background Canvas with mouse click
        self.background.bind("<B1-Motion>",self.paint) 


# Functions are defined here

# Paint Function for Drawing the lines on Canvas
    def paint(self,event):       
        x1,y1 = (event.x-2), (event.y-2)  
        x2,y2 = (event.x+2), (event.y+2)  

        self.background.create_oval(x1,y1,x2,y2,fill=self.pointer,outline=self.pointer,width=self.pointer_size.get())

# Function for choosing the color of pointer  
    def select_color(self,col):
        self.pointer = col

# Function for defining the eraser
    def eraser(self):
        self.pointer= self.erase

# Function for choosing the background color of the Canvas    
    def canvas_color(self):
        color=colorchooser.askcolor()
        self.background.configure(background=color[1])
        self.erase= color[1]

# Function for saving the image file in Local Computer
    def save_drawing(self):
        try:
            # self.background update()
#             file_ss =filedialog.asksaveasfilename(defaultextension='jpg')
            #print(file_ss)
            x = self.root.winfo_rootx() + self.background.winfo_x() + 60
            #print(x, self.background.winfo_x())
            y = self.root.winfo_rooty() + self.background.winfo_y() + 55
            #print(y)
            x1= x + self.background.winfo_width() - 3*self.background.winfo_x() -90
            #print(x1)
            y1= y + self.background.winfo_height() - 2*self.background.winfo_y() -40
            #print(y1)
            ImageGrab.grab().crop((x , y, x1, y1)).save("temp.jpg")
#             messagebox.showinfo('Please wait')
            identify()
        except:
            print("Error. Please wait")

if __name__ =="__main__":
    root = Tk()
    p= Draw(root)
    root.mainloop()

<class 'numpy.ndarray'>
5
<class 'numpy.ndarray'>
3
<class 'numpy.ndarray'>
3
<class 'numpy.ndarray'>
6
<class 'numpy.ndarray'>
3
<class 'numpy.ndarray'>
0
